In [37]:
import os
import ember
import lightgbm as lgb
import shap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

binary_name = "DUAL_COUGH" #set this to the binary name, put the ones u want to test in /binaries

data_dir = "../ember2018/"

In [38]:
# ~4 mns
binary_location = f"/workspaces/torment-nexus/binaries/{binary_name}"

lgbm_model = ember.train_model("/workspaces/torment-nexus/ember2018")
emberdf = ember.read_metadata(data_dir)
X_train, y_train, X_test, y_test = ember.read_vectorized_features(data_dir, feature_version=2)
lgbm_model = lgb.Booster(model_file=os.path.join(data_dir, "ember_model_2018.txt"))
lgbm_model.params['objective'] = 'regression' # dunno why this never gets set but its needed for shap

[LightGBM] [Info] Number of positive: 300000, number of negative: 300000
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 2.989532 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 212057
[LightGBM] [Info] Number of data points in the train set: 600000, number of used features: 2333
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [39]:
extractor1 = ember.PEFeatureExtractor(1)
extractor2 = ember.PEFeatureExtractor(2)

file_data = open(binary_location, "rb").read()

feature_vector = extractor2.feature_vector(file_data)
raw_features = extractor2.raw_features(file_data)

lief error:  This file is not a PE binary
lief error:  This file is not a PE binary


In [60]:
feature_names = []
for a in raw_features.keys():
    feature_names.append(a)
    if type(raw_features.get(a, None)) is dict: 
        for b in raw_features[a]:
            feature_names.append(b)

feature_names

['sha256',
 'histogram',
 'byteentropy',
 'strings',
 'numstrings',
 'avlength',
 'printabledist',
 'printables',
 'entropy',
 'paths',
 'urls',
 'registry',
 'MZ',
 'general',
 'size',
 'vsize',
 'has_debug',
 'exports',
 'imports',
 'has_relocations',
 'has_resources',
 'has_signature',
 'has_tls',
 'symbols',
 'header',
 'coff',
 'optional',
 'section',
 'entry',
 'sections',
 'imports',
 'exports',
 'datadirectories']

In [40]:
explainer = shap.TreeExplainer(lgbm_model, feature_names=raw_features)
shap_values = explainer(pd.DataFrame(X_train[:10]))
shap_values_single = explainer(np.array([feature_vector], dtype=np.float32))

In [41]:
shap.initjs()
shap.force_plot(shap_values)

In [42]:
shap.force_plot(shap_values_single)

In [43]:
"""
You can see in the above graph that all of the features still have "Feature XXXX" instead of an actual name.
Forunately, there is something resembling a feature name that you can see below
"""
np.array(raw_features)

array({'sha256': '25d3e4916bbf90e25c9f303eb23a550fa5f9bdb175e592782401dca6823f2d06', 'histogram': [2989892, 431415, 332850, 156179, 182924, 222024, 103800, 92645, 174601, 84785, 93647, 63049, 57778, 51835, 42659, 292519, 106398, 43495, 32708, 41597, 40272, 32161, 26313, 25874, 65437, 26017, 23706, 25276, 21933, 24006, 21014, 66399, 68221, 21863, 18934, 14294, 653981, 23260, 19495, 14686, 49436, 44167, 28909, 13298, 17682, 13399, 114354, 12685, 57881, 116567, 39879, 31193, 36889, 30424, 33218, 30426, 52406, 61598, 13308, 27047, 16374, 28840, 10562, 15254, 78499, 49833, 33925, 61776, 376898, 41250, 38529, 36484, 828972, 87635, 34263, 31824, 210413, 52423, 32021, 31852, 103142, 33799, 37392, 33184, 114253, 30338, 29335, 32277, 47242, 29353, 30072, 14502, 63388, 10901, 10451, 32781, 38585, 39272, 31685, 42369, 39531, 51209, 81101, 90587, 46360, 43369, 27743, 29520, 130964, 35522, 49503, 95102, 51383, 29377, 48384, 59874, 70986, 69090, 35435, 29421, 45100, 27399, 50521, 13116, 31164, 13335,

In [48]:
shap.plots.waterfall(shap_values_single[0], max_display=5)

KeyError: 0

In [ ]:
shap_values_single.display_data = None

ValueError: arrays must all be same length